## Composability (WIP)

`bodhilib` library is designed with composability in mind. It takes many ideas from strict functional languages like `Haskell` to design and implement its interface.

Using the bodhilib library, you can simplify the ingestion phase of your RAG process as follows:

# Setup

1. Ensure bodhilib is installed, along with LLM plugin (`bodhiext.openai`), Embedder plugin (`bodhiext.sentence_transformers`), VectorDB plugin (`bodhiext.qdrant`).
1. Ensure `fn.py` is installed for the functional composition methods.
1. Ensure OPENAI_API_KEY is set in environment variables.

In [1]:
!pip install -q bodhilib bodhiext.openai bodhiext.sentence_transformers bodhiext.qdrant fn.py

In [3]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key:")

In [6]:
# load the components
from bodhilib import get_llm, get_data_loader, get_embedder, get_vector_db, get_splitter

data_loader = get_data_loader("file")
spitter = get_splitter("text_splitter")
embedder = get_embedder("sentence_transformers")
vector_db = get_vector_db("qdrant", location=":memory:")

In [8]:
from fn import Stream

f = Stream()

In [ ]:
from fn import F # fn.py library

data_loader.add_resource(dir="./data")
data = data_loader.load()

result = F(data_loader.load) 
    >> F(splitter.split) 
    >> F(embedder.embed) 
    >> F(vector_db.upsert)

And to query your VectorDB, you can compose it like:

In [ ]:
query = "Who is the CEO of SpaceX?"
template = get_prompt_source("bodhiprompts").find("extractive_qna")

answer = (
    query
    >> F(embedder.embed)
    >> F(partial(vector_db.query, "articles_collection"))
    >> F(lambda nodes: [node.text for node in nodes])
    >> F(lambda nodes: {"context": "\n\n".join(nodes)})
    >> F(partial(template.to_prompt, query = query))
    >> F(llm.generate)
)
